## Install Required Packages

In [ ]:
!pip install librosa torchaudio openai-whisper transformers \
    matplotlib seaborn scikit-learn accelerate
!apt-get install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.2 MB/

## Setup Project Structure

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Create project folder
project_dir = "/content/drive/MyDrive/multimodal_emotion_recognition"
!mkdir -p {project_dir}/data {project_dir}/spectrograms {project_dir}/transcripts {project_dir}/mfccs
data_dir = os.path.join(project_dir, "data")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Unzip and Check Extracted Files

In [ ]:
!unzip -q "{data_dir}/archive.zip" -d "{data_dir}"

In [ ]:
import glob

# Recursively get all WAV files
audio_files = sorted(glob.glob(os.path.join(data_dir, '**', '*.wav'), recursive=True))

print(f"Found {len(audio_files)} WAV files.")
print(audio_files[:5])  # preview a few

Found 2880 WAV files.
['/content/drive/MyDrive/multimodal_emotion_recognition/data/Actor_01/03-01-01-01-01-01-01.wav', '/content/drive/MyDrive/multimodal_emotion_recognition/data/Actor_01/03-01-01-01-01-02-01.wav', '/content/drive/MyDrive/multimodal_emotion_recognition/data/Actor_01/03-01-01-01-02-01-01.wav', '/content/drive/MyDrive/multimodal_emotion_recognition/data/Actor_01/03-01-01-01-02-02-01.wav', '/content/drive/MyDrive/multimodal_emotion_recognition/data/Actor_01/03-01-02-01-01-01-01.wav']


## Generate and Save Spectrograms for CNN Input

In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import glob

# Set directories
project_dir = "/content/drive/MyDrive/multimodal_emotion_recognition"
data_dir = os.path.join(project_dir, "data")
spec_dir = os.path.join(project_dir, "spectrograms")
os.makedirs(spec_dir, exist_ok=True)

# Load audio files
audio_files = sorted(glob.glob(os.path.join(data_dir, '**', '*.wav'), recursive=True))

# Emotion label map
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

def parse_emotion(filename):
    parts = filename.split("-")
    return emotion_map.get(parts[2], "unknown")

# Generate and save spectrograms
for file in tqdm(audio_files, desc="Generating spectrograms"):
    try:
        y, sr = librosa.load(file, sr=22050)
        spec = librosa.feature.melspectrogram(y=y, sr=sr)
        spec_db = librosa.power_to_db(spec, ref=np.max)

        # Save as image
        emotion = parse_emotion(os.path.basename(file))
        base = os.path.splitext(os.path.basename(file))[0]
        out_path = os.path.join(spec_dir, f"{emotion}_{base}.png")

        plt.figure(figsize=(3, 3))
        librosa.display.specshow(spec_db, sr=sr, cmap='magma')
        plt.axis('off')
        plt.tight_layout(pad=0)
        plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
        plt.close()
    except Exception as e:
        print(f"⚠️ Error processing {file}: {e}")


Generating spectrograms: 100%|██████████| 2880/2880 [07:15<00:00,  6.61it/s]


## Generate and Save Transcripts for the RNN

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-kemy_58g
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-kemy_58g
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
import whisper
model = whisper.load_model("base")

In [ ]:
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

In [ ]:
from google.colab import drive
import os
import json

drive.mount('/content/drive')
audio_folder = "/content/drive/MyDrive/multimodal_emotion_recognition/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
output = []

for root, dirs, files in os.walk(audio_folder):
    for fname in files:
        if fname.endswith(".wav"):
            path = os.path.join(root, fname)
            result = model.transcribe(path)
            label = emotion_map[fname.split("-")[2]]

            output.append({
                "filename": fname,
                "transcript": result["text"],
                "label": label
            })

# Save transcripts as JSON
with open("/content/drive/MyDrive/multimodal_emotion_recognition/transcripts/transcripts.json", "w") as f:
    json.dump(output, f, indent=2)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not sup